##XG BOOST

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('../clean_train.csv')
df_test = pd.read_csv('../clean_test.csv')

df.head()

,id,gps_height,longitude,latitude,region_code,district_code,population,payment,status_group,recordDay,...,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
0,69572,1390.0,34.938093,-9.856322,11,5,109.0,1.0,functional,14,...,1,0,0,0,1,0,0,0,0,0
1,8776,1399.0,34.698766,-2.147466,20,2,280.0,0.0,functional,6,...,0,1,0,0,1,0,0,0,0,0
2,34310,686.0,37.460664,-3.821329,21,4,250.0,1.0,functional,25,...,0,1,0,0,0,1,0,0,0,0
3,67743,263.0,38.486161,-11.155298,90,63,58.0,0.0,non functional,28,...,1,0,0,0,0,1,0,0,0,0
4,19728,1300.0,31.130847,-1.825359,18,1,250.0,0.0,functional,13,...,0,1,0,0,1,0,0,0,0,0


In [3]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

from sklearn import tree


In [4]:
df.columns.difference(df_test.columns)

Index(['extraction_type_other - mkulima/shinyanga', 'scheme_management_None',
       'status_group'],
      dtype='object')

In [5]:
df.drop(columns = ['extraction_type_other - mkulima/shinyanga','scheme_management_None'], inplace=True)

In [6]:
df.shape, df_test.shape

((59400, 63), (14850, 62))

In [7]:

df['status_group']=df['status_group'].replace("functional",2)
df['status_group']=df['status_group'].replace("non functional",1)
df['status_group']=df['status_group'].replace("functional needs repair",0)

### Modeling

In [8]:

from sklearn import model_selection
from sklearn.model_selection import train_test_split

X = df.drop(columns=['status_group'], axis=1)
y = df['status_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=101)

print('Training set: ', X_train.shape, y_train.shape)
print('Testing set: ', X_test.shape, y_test.shape)

Training set:  (41580, 62) (41580,)
Testing set:  (17820, 62) (17820,)


In [9]:
xgb = XGBClassifier(n_estimators=100,reg_alpha=1,reg_lambda=0.01)
xgb.fit(X_train,y_train)
predictions = xgb.predict(X_test)

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 295  216  819]
 [  68 4907 1965]
 [ 106  764 8680]]
              precision    recall  f1-score   support

           0       0.63      0.22      0.33      1330
           1       0.83      0.71      0.77      6940
           2       0.76      0.91      0.83      9550

    accuracy                           0.78     17820
   macro avg       0.74      0.61      0.64     17820
weighted avg       0.78      0.78      0.77     17820



In [10]:
print('Train set Accuracy: ', metrics.accuracy_score(y_train, xgb.predict(X_train)))
print('Test set Accuracy: ', metrics.accuracy_score(y_test, predictions))

Train set Accuracy:  0.8441798941798941
Test set Accuracy:  0.7790123456790123


### Submission File for DrivenData

Create prediction on test data and create dataframe for competition submission.

In [26]:
X2 = df_test.values
prediction_test = xgb.predict(X2)

df_pred = pd.DataFrame(prediction_test)

df_pred.columns=['status_group']

df_pred['status_group']=df_pred['status_group'].replace(2,"functional")
df_pred['status_group']=df_pred['status_group'].replace(1,"non functional")
df_pred['status_group']=df_pred['status_group'].replace(0,"functional needs repair")

iddf= pd.read_csv('../testSet-indep.csv')
df = pd.concat([iddf['id'], df_pred], axis=1)


In [57]:
df.to_csv('../submissions/Submission_xgb.csv', index=False)